In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta, time
os.environ['TZ'] ='America/New_York'

import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from ib_async import *
util.startLoop()

### Data Prep for Inferencing

In [2]:
ib = IB()
ib.connect(port=4002, clientId=2)

<IB connected to 127.0.0.1:4002 clientId=2>

In [3]:
contract = Contract(symbol='META', secType='STK', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

[Contract(secType='STK', conId=107113386, symbol='META', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='META', tradingClass='NMS')]

In [4]:
contract_vix = Contract(symbol= 'VIX', secType = 'IND',exchange = 'CBOE', currency='USD')
ib.qualifyContracts(contract_vix)

[Contract(secType='IND', conId=13455763, symbol='VIX', exchange='CBOE', currency='USD', localSymbol='VIX')]

In [5]:
contract_nas = Contract(symbol= 'TQQQ', secType = 'STK',exchange = 'SMART', currency='USD')
ib.qualifyContracts(contract_nas)

[Contract(secType='STK', conId=72539702, symbol='TQQQ', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='TQQQ', tradingClass='NMS')]

In [6]:
class Data_Request :

    """
    Returns PD data containning equity price with specific time control

    Returns:
        pd: information regarding the contract of interst.
    """
    
    def __init__(self, list_days, contract, barsize, duration, *,specific_time= None, specific_pick_time= None):

        self.list_days = list_days
        self.contract = contract
        #self.init_date = init_date
        self.barsize = barsize
        self.duration = duration
        self.specific_time = specific_time # cut off time . Time object: time(14, 30, 0)
        self.specific_pick_time= specific_pick_time

    def D_request(self):

        his_df =pd.DataFrame()
        
        for i in range(len(self.list_days)):
            init_date=self.list_days[i]
            bars = ib.reqHistoricalData(self.contract, init_date, barSizeSetting=self.barsize, durationStr=self.duration, whatToShow="TRADES", useRTH=True)
            add_his_df = pd.DataFrame(bars)
            add_his_df['date'] = pd.to_datetime(add_his_df['date'])
            print(f'retriveing data for {init_date}')

            if self.specific_time is not None:
                cut_off = datetime.combine(self.list_days[i],self.specific_time)  # need logic if specific_time is defined
                pd_cut_off = pd.to_datetime(cut_off).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] > pd_cut_off]
            
            if self.specific_pick_time is not None:
                pick_time = datetime.combine(self.list_days[i],self.specific_pick_time)  # need logic if specific_time is defined
                pd_pick_time = pd.to_datetime(pick_time).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] == pd_pick_time]

            
            his_df = pd.concat([his_df, add_his_df], ignore_index=True)
            his_df['Date_Only'] = his_df['date'].dt.date
            print(len(his_df))

        
        return his_df


In [7]:
# picking out Friday and before specific-time data

#specific_time = time(14, 30, 0)
def data_filter(df_initial, list_days, specific_time_before = None, specific_time_after= None, on_time= None):
    his_df = pd.DataFrame()
    for i in range(len(list_days)):
        init_date=list_days[i]
        if specific_time_before is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time < specific_time_before)]
           
            
        if specific_time_after is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time >= specific_time_after)]

        if on_time is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time == on_time)]

        if (specific_time_before is None and specific_time_after is None) and on_time is None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date)]
        
        print(len(his_df))
        print(f'retrieving {init_date}')
        per_proc = np.round(i+1/len(list_days)*100, decimals=1)
        print(f'{per_proc} percent is done')
        his_df = pd.concat([his_df, add_his_df], ignore_index=True)
        his_df.drop_duplicates(subset=None, keep='first', inplace=True)
        his_df['Date_Only'] = his_df['date'].dt.date
    return his_df

In [50]:
def get_fridays_in_range(start_date, end_date):
    """
    Returns a list of dates that are Fridays within the given date range.

    Args:
        start_date (date): The start date of the range.
        end_date (date): The end date of the range.

    Returns:
        list: A list of date objects representing Fridays within the range.
    """
    fridays = []
    all_dates = []
    current_date = start_date
    all_current_date = start_date

    # Adjust to the first Friday
    days_until_friday = (4 - current_date.weekday()) % 7
    current_date += timedelta(days=days_until_friday)

    while current_date <= end_date:
        fridays.append(current_date)
        current_date += timedelta(days=7) # Move to the next Friday

    while all_current_date <= end_date:
        all_dates.append(all_current_date)
        all_current_date += timedelta(days=1)
    
    return fridays, all_dates

# Example Usage
start_date = date(2024, 1, 1)
end_date = date(2025, 4, 1)

fridays, all_dates = get_fridays_in_range(start_date, end_date)
# removing Non-trading Holidays
fridays.remove(date(2024, 3, 29))
fridays.remove(date(2024, 11, 29))

In [9]:
now =datetime.now()
print(now.date())

2025-04-02


### Trainning Data Set Collection ###

In [14]:
barsize ="1 min"
duration = "1 D"
DR_stock_meta = Data_Request(all_dates, contract, barsize, duration)

In [44]:
DR_IND_VIX = Data_Request(all_dates, contract_vix, barsize, duration)
DR_STK_TQQQ = Data_Request(all_dates, contract_nas, barsize, duration)
df_all_days_VIX = DR_IND_VIX.D_request()
df_all_days_TQQQ = DR_STK_TQQQ.D_request()

retriveing data for 2024-01-01
810
retriveing data for 2024-01-02
1620
retriveing data for 2024-01-03
2430
retriveing data for 2024-01-04
3240
retriveing data for 2024-01-05
4050
retriveing data for 2024-01-06
4860
retriveing data for 2024-01-07
5670
retriveing data for 2024-01-08
6480
retriveing data for 2024-01-09
7290
retriveing data for 2024-01-10
8100
retriveing data for 2024-01-11
8910
retriveing data for 2024-01-12
9720
retriveing data for 2024-01-13
10530
retriveing data for 2024-01-14
11340
retriveing data for 2024-01-15
12150
retriveing data for 2024-01-16
12960
retriveing data for 2024-01-17
13770
retriveing data for 2024-01-18
14580
retriveing data for 2024-01-19
15390
retriveing data for 2024-01-20
16200
retriveing data for 2024-01-21
17010
retriveing data for 2024-01-22
17820
retriveing data for 2024-01-23
18630
retriveing data for 2024-01-24
19440
retriveing data for 2024-01-25
20250
retriveing data for 2024-01-26
21060
retriveing data for 2024-01-27
21870
retriveing dat

In [15]:
df_all_days_stock = DR_stock_meta.D_request()

retriveing data for 2024-01-01
390
retriveing data for 2024-01-02
780
retriveing data for 2024-01-03
1170
retriveing data for 2024-01-04
1560
retriveing data for 2024-01-05
1950
retriveing data for 2024-01-06
2340
retriveing data for 2024-01-07
2730
retriveing data for 2024-01-08
3120
retriveing data for 2024-01-09
3510
retriveing data for 2024-01-10
3900
retriveing data for 2024-01-11
4290
retriveing data for 2024-01-12
4680
retriveing data for 2024-01-13
5070
retriveing data for 2024-01-14
5460
retriveing data for 2024-01-15
5850
retriveing data for 2024-01-16
6240
retriveing data for 2024-01-17
6630
retriveing data for 2024-01-18
7020
retriveing data for 2024-01-19
7410
retriveing data for 2024-01-20
7800
retriveing data for 2024-01-21
8190
retriveing data for 2024-01-22
8580
retriveing data for 2024-01-23
8970
retriveing data for 2024-01-24
9360
retriveing data for 2024-01-25
9750
retriveing data for 2024-01-26
10140
retriveing data for 2024-01-27
10530
retriveing data for 2024-01-

In [45]:
df_all_days_stock.to_csv('master_stock_info.csv', index=False)
df_all_days_stock.to_pickle('master_stock_info.pkl')

df_all_days_VIX.to_csv('master_vix_info.csv', index=False)
df_all_days_VIX.to_pickle('master_vix_info.pkl')

df_all_days_TQQQ.to_csv('master_tqqq_info.csv', index=False)
df_all_days_TQQQ.to_pickle('master_tqqq_info.pkl')

,date,open,high,low,close,volume,average,barCount,Date_Only
0,2023-12-29 09:30:00-05:00,358.99,360.00,358.99,359.88,155021.0,359.308,510,2023-12-29
1,2023-12-29 09:31:00-05:00,359.82,359.82,359.27,359.71,38832.0,359.594,254,2023-12-29
2,2023-12-29 09:32:00-05:00,359.69,359.72,358.33,358.48,67898.0,359.113,374,2023-12-29
3,2023-12-29 09:33:00-05:00,358.62,358.72,357.66,357.94,30127.0,358.207,180,2023-12-29
4,2023-12-29 09:34:00-05:00,357.83,357.89,357.25,357.28,24586.0,357.559,153,2023-12-29
...,...,...,...,...,...,...,...,...,...
176965,2025-04-01 15:55:00-04:00,586.51,587.37,585.49,585.74,51718.0,586.359,386,2025-04-01
176966,2025-04-01 15:56:00-04:00,585.78,586.14,585.44,586.02,55195.0,585.823,399,2025-04-01
176967,2025-04-01 15:57:00-04:00,586.10,586.20,585.74,585.93,43672.0,585.957,339,2025-04-01
176968,2025-04-01 15:58:00-04:00,585.95,586.35,585.82,586.31,63359.0,586.068,517,2025-04-01


In [46]:
cut_time = time(14, 30, 0)

In [103]:

def data_sparcing_construc(df, sp_day, *, cut_time = None):
    #df_group = df.groupby("Date_Only").std(numeric_only=True)
    df_filter_prior = data_filter(df, sp_day, specific_time_before = cut_time )
    df_group_prior = df_filter_prior.groupby("Date_Only").std(numeric_only=True)
    
    df_filter_after = data_filter(df, sp_day, specific_time_after = cut_time )
    df_group_after = df_filter_after.groupby("Date_Only").std(numeric_only=True)


    df_group_train =pd.DataFrame()
    df_group_train['Target_std'] = pd.DataFrame(df_group_after[['open','high','low','close']].max(axis=1))
    df_group_train['prior_std'] = pd.DataFrame(df_group_prior[['open','high','low','close']].max(axis=1))
    df_group_train['prior_Vol'] = pd.DataFrame(df_filter_prior.groupby("Date_Only").mean()['volume'])
    
    df_initial = df_filter_prior
    new_time = time(cut_time.hour, cut_time.minute -1)
    df_group_train['cut_open'] = df_initial[(df_initial['date'].dt.time == new_time)]['open'].values
    
    cut_name =['cut_1hr','cut_2hr','cut_3hr']
    for i in range(3):
        prior = time(cut_time.hour - (i+1), cut_time.minute)
        df_group_train[cut_name[i]] = df_initial[(df_initial['date'].dt.time == new_time)]['open'].values- df_initial[(df_initial['date'].dt.time == prior)]['open'].values


    
    return df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after
    

In [104]:
df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after = data_sparcing_construc(df_all_days_stock, fridays, cut_time= cut_time)

0
retrieving 2024-01-05
1.6 percent is done
300
retrieving 2024-01-12
2.6 percent is done
600
retrieving 2024-01-19
3.6 percent is done
900
retrieving 2024-01-26
4.6 percent is done
1200
retrieving 2024-02-02
5.6 percent is done
1500
retrieving 2024-02-09
6.6 percent is done
1800
retrieving 2024-02-16
7.6 percent is done
2100
retrieving 2024-02-23
8.6 percent is done
2400
retrieving 2024-03-01
9.6 percent is done
2700
retrieving 2024-03-08
10.6 percent is done
3000
retrieving 2024-03-15
11.6 percent is done
3300
retrieving 2024-03-22
12.6 percent is done
3600
retrieving 2024-04-05
13.6 percent is done
3900
retrieving 2024-04-12
14.6 percent is done
4200
retrieving 2024-04-19
15.6 percent is done
4500
retrieving 2024-04-26
16.6 percent is done
4800
retrieving 2024-05-03
17.6 percent is done
5100
retrieving 2024-05-10
18.6 percent is done
5400
retrieving 2024-05-17
19.6 percent is done
5700
retrieving 2024-05-24
20.6 percent is done
6000
retrieving 2024-05-31
21.6 percent is done
6300
re

In [108]:
df_group_train_vix, *_  = data_sparcing_construc(df_all_days_VIX, fridays, cut_time= cut_time)
df_group_train_tqqq, *_ = data_sparcing_construc(df_all_days_TQQQ, fridays, cut_time= cut_time)

0
retrieving 2024-01-05
1.6 percent is done
720
retrieving 2024-01-12
2.6 percent is done
1440
retrieving 2024-01-19
3.6 percent is done
2160
retrieving 2024-01-26
4.6 percent is done
2880
retrieving 2024-02-02
5.6 percent is done
3600
retrieving 2024-02-09
6.6 percent is done
4320
retrieving 2024-02-16
7.6 percent is done
5040
retrieving 2024-02-23
8.6 percent is done
5760
retrieving 2024-03-01
9.6 percent is done
6480
retrieving 2024-03-08
10.6 percent is done
7200
retrieving 2024-03-15
11.6 percent is done
7920
retrieving 2024-03-22
12.6 percent is done
8640
retrieving 2024-04-05
13.6 percent is done
9360
retrieving 2024-04-12
14.6 percent is done
10080
retrieving 2024-04-19
15.6 percent is done
10800
retrieving 2024-04-26
16.6 percent is done
11520
retrieving 2024-05-03
17.6 percent is done
12240
retrieving 2024-05-10
18.6 percent is done
12960
retrieving 2024-05-17
19.6 percent is done
13680
retrieving 2024-05-24
20.6 percent is done
14400
retrieving 2024-05-31
21.6 percent is don

In [130]:
df_final_joninted_vix=pd.DataFrame()
df_final_joninted_all=pd.DataFrame()

df_group_train_vix.columns =['vix_std','vix_prior_std','vix_Vol','vix_cut_open','vix_cut_1hr','vix_cut_2hr','vix_cut_3hr']
df_final_jointed_vix = pd.concat([df_group_train,df_group_train_vix], axis=1)
df_group_train_tqqq.columns =['tqqq_std','tqqq_prior_std','tqqq_Vol','tqqq_cut_open','tqqq_cut_1hr','tqqq_cut_2hr','tqqq_cut_3hr']
df_final_jointed_all = pd.concat([df_final_jointed_vix,df_group_train_tqqq], axis=1)
df_final_jointed_all.drop(['vix_std','vix_Vol','tqqq_std'], axis=1, inplace=True)

In [131]:
df_final_jointed_all

,Target_std,prior_std,prior_Vol,cut_open,cut_1hr,cut_2hr,cut_3hr,vix_prior_std,vix_cut_open,vix_cut_1hr,vix_cut_2hr,vix_cut_3hr,tqqq_prior_std,tqqq_Vol,tqqq_cut_open,tqqq_cut_1hr,tqqq_cut_2hr,tqqq_cut_3hr
Date_Only,,,,,,,,,,,,,,,,,,
2024-01-05,0.650053,1.058044,18218.376667,351.89,0.70,0.97,-0.11,0.463383,13.53,0.10,0.03,-0.17,0.301283,199340.133333,46.10,0.05,0.18,-0.34
2024-01-12,0.629948,0.993937,29812.156667,375.53,0.89,1.75,-0.75,0.109005,12.55,-0.02,-0.03,-0.15,0.223571,156363.693333,50.40,0.06,0.29,-0.05
2024-01-19,0.509282,1.115984,28760.476667,383.97,0.83,2.83,2.65,0.097359,13.41,-0.28,-0.22,-0.08,0.651070,206204.083333,54.46,0.71,1.27,1.55
2024-01-26,0.813494,1.238336,18011.023333,395.97,0.11,-0.05,-0.30,0.176661,13.30,0.02,0.01,-0.04,0.356931,146601.786667,55.44,-0.11,-0.63,-0.75
2024-02-02,2.322338,4.712763,156749.593333,478.94,2.13,-2.85,1.92,0.108060,13.81,-0.24,-0.22,-0.11,0.611902,229504.093333,57.40,0.33,-0.04,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,2.284799,3.457627,19624.563333,657.29,-1.04,-3.43,-1.62,0.465128,20.65,-0.91,-1.07,-0.07,0.767429,200316.643333,71.61,0.12,-0.97,-0.34
2025-03-07,1.311185,8.718339,35741.483333,626.00,10.33,19.82,18.71,0.575226,23.47,-0.26,-1.38,-2.53,1.292159,314475.686667,67.48,1.73,3.52,3.54
2025-03-14,0.758964,2.965842,18741.093333,605.71,0.88,0.58,2.23,0.894907,21.74,-0.48,-0.61,-0.98,0.581809,234059.553333,61.26,-0.16,0.19,-0.10


### INFERENCING###